In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import os
import sys
# sys.path.insert(1, os.path.join(sys.path[0], '..'))
os.chdir('..')

from dataset import DataSet
from model_part import conv2d
from model_part import fc

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
today = datetime.strftime(datetime.now(), '%d%m%y')
flags.DEFINE_integer('batch_size', 8, 'Batch size')
flags.DEFINE_string('train_file', 'train.csv', 'Train file')

In [40]:
def decoder_model(inputs, reuse=False, trainable=True, debug=False, keep_drop=0.5):
    # inputs.get_shape() = TensorShape([Dimension(8), Dimension(55), Dimension(74), Dimension(1)])
    if debug:
        print('%s \t\t\t\t%s' % ('inputs', inputs.get_shape()))
        
    net1 = conv2d('coarse1', inputs, [11, 11, 3, 96], [96], [1, 4, 4, 1],
                 padding='VALID', reuse=reuse, trainable=trainable)
    if debug:
        print('%s \t\t%s' % (net1.name, net1.get_shape()))
    net = tf.nn.max_pool(net1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='VALID', name='pool1')
    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))
        
    net2 = conv2d('coarse2', net, [5, 5, 96, 256], [256], [1, 1, 1, 1],
                 padding='VALID', reuse=reuse, trainable=trainable)
    if debug:
        print('%s \t\t%s' % (net2.name, net2.get_shape()))
    net = tf.nn.max_pool(net2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='SAME', name='pool2')
    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = conv2d('coarse3', net, [3, 3, 256, 384], [384], [1, 1, 1, 1],
                 padding='VALID', reuse=reuse, trainable=trainable)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net4 = conv2d('coarse4', net, [3, 3, 384, 384], [384], [1, 1, 1, 1],
                 padding='VALID', reuse=reuse, trainable=trainable)
    if debug:
        print('%s \t\t%s' % (net4.name, net4.get_shape()))
    net = conv2d('coarse5', net4, [3, 3, 384, 256], [256], [1, 1, 1, 1],
                 padding='VALID', reuse=reuse, trainable=trainable)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = fc('coarse6', net, [6*10*256, 4096], [4096], reuse=reuse,
             trainable=trainable)
    
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.nn.dropout(net, keep_drop)

    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = fc('coarse7', net, [4096, 6*10*256], [6*10*256], reuse=reuse,
             trainable=trainable)
    if debug:
            print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.nn.dropout(net, keep_drop)

    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.reshape(net, [-1, 6, 10, 256])

    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[8, 12],
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug:
        print('%s \t%s' % (net.name, net.get_shape()))
    net = conv2d('up1', net, [3, 3, 256, 384], [384], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable)
    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))
    
    # residual
    net = net + net4
    if debug:
        print('%s \t\t\t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[23, 32],
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug:
        print('%s \t%s' % (net.name, net.get_shape()))
    net = conv2d('up2', net, [3, 3, 384, 256], [256], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable)
    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))
    
    # residual
    net = net + net2
    
    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[55, 74],
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug:
        print('%s \t%s' % (net.name, net.get_shape()))
    net = conv2d('up3', net, [3, 3, 256, 96], [96], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable)
    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))
    
    # residual
    net = net + net1
    
    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[228, 304],
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug:
        print('%s \t%s' % (net.name, net.get_shape()))
    net = conv2d('up4', net, [3, 3, 96, 1], [1], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable)
    
    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))

    return net

In [41]:
with tf.Graph().as_default():
    dataset = DataSet(FLAGS.batch_size)
    images, depths, _ = dataset.csv_inputs(FLAGS.train_file,
                                               target_size=[228, 304])
    keep_hidden = tf.placeholder(tf.float32)
    encoder_model = decoder_model(images, keep_hidden, trainable=False, debug=True)

inputs 				(8, 228, 304, 3)
coarse1/coarse1:0 		(8, 55, 74, 96)
pool1:0 			(8, 27, 36, 96)
coarse2/coarse2:0 		(8, 23, 32, 256)
pool2:0 			(8, 12, 16, 256)
coarse3/coarse3:0 		(8, 10, 14, 384)
coarse4/coarse4:0 		(8, 8, 12, 384)
coarse5/coarse5:0 		(8, 6, 10, 256)
coarse6/coarse6:0 		(8, 4096)
dropout/mul:0 			(8, 4096)
coarse7/coarse7:0 		(8, 15360)
dropout_1/mul:0 		(8, 15360)
Reshape_1:0 			(8, 6, 10, 256)
ResizeNearestNeighbor:0 	(8, 8, 12, 256)
up1/up1:0 			(8, 8, 12, 384)
add:0 				(8, 8, 12, 384)
ResizeNearestNeighbor_1:0 	(8, 23, 32, 384)
up2/up2:0 			(8, 23, 32, 256)
add_1:0 			(8, 23, 32, 256)
ResizeNearestNeighbor_2:0 	(8, 55, 74, 256)
up3/up3:0 			(8, 55, 74, 96)
add_2:0 			(8, 55, 74, 96)
ResizeNearestNeighbor_3:0 	(8, 228, 304, 96)
up4/up4:0 			(8, 228, 304, 1)


In [21]:
with tf.Graph().as_default():
    dataset = DataSet(FLAGS.batch_size)
    images, depths, _ = dataset.csv_inputs(FLAGS.train_file,
                                               target_size=[228, 304])
    keep_hidden = tf.placeholder(tf.float32)
    encoder_model = decoder_model(images, keep_hidden, trainable=False, debug=True)

inputs 				(8, 228, 304, 3)
coarse1/coarse1:0 		(8, 55, 74, 96)
pool1:0 			(8, 27, 36, 96)
coarse2/coarse2:0 		(8, 23, 32, 256)
pool2:0 			(8, 12, 16, 256)
coarse3/coarse3:0 		(8, 10, 14, 384)
coarse4/coarse4:0 		(8, 8, 12, 384)
coarse5/coarse5:0 		(8, 6, 10, 256)
coarse6/coarse6:0 		(8, 4096)
dropout/mul:0 			(8, 4096)
coarse7/coarse7:0 		(8, 15360)
dropout_1/mul:0 		(8, 15360)
Reshape_1:0 			(8, 6, 10, 256)
ResizeNearestNeighbor:0 	(8, 8, 12, 256)
up1/up1:0 			(8, 8, 12, 384)
